# AI Community @ Семинар № 8, весна 2018
## Texture synthesis and Style transfer.

Перед тем как приступать непосредственно к переносу стилей, научимся генерировать текстуры из данного шаблона.   
Пусть у нас есть следующий фрагмент
<img src='images/pebbles.jpg'>
Как мы помним, каждый сверточный слой выделяет на изображении определенные низкоуровневые признаки. Прогоним данное изображение через глубокую сверточную сеть. При этом будем сохранять значения полученные после очередного слоя. Они будут иметь размерность $(N \times C \times H \times W)$. Сейчас будем рассматривать один пример, т.е. размерность будет $(C \times H \times W)$.  
N - число примеров(в данном случае всегда 1), C - количество карт признаков, H, W - ширина и высота изображения соответсвенно.  
<img src='images/featuremap.png'>
<div style="text-align: right"><font size="2">Изображение из презентации [2]</font></div>   
Данный тензор можно представить в виде $H \times W$ векторов размерности $C$.

Выберем среди них два произвольных вектора $a$ и $b$ и посчитаем для них 
[матрицу Грама](https://en.wikipedia.org/wiki/Gramian_matrix)
    
$$G_{ij} = a_{i} * b_{j}$$

Она будет приближенно отражать ковариацию между занчениями результатов каждого фильтра. Наша цель - получить изображение в том же распределении. Равенство ковариаций между результатами различных фильтров будет давать искомый эффект.    
Почему именно матрица Грама? Потому что, как обычно в глубоком обучении и происходит: попробовали -> заработало. Также, как мы увидим ниже, ее очень просто и эффективно считать.    
Теперь посчитаем такие матрицы для всех возможных пар векторов и усредним полученные значения поделив на $H \times W$. В результате мы получим $(C \times C)$ матрицу.

Как считать ее на практике:   
Обозначим за $F$ тензор признаков вытянутый в длину так, чтобы в результате размерность стала равна $(C \times (H * W))$. Тогда   
$$G = FF^T$$

Вспомним как тренируются нейронные сети. Мы считаем $\frac{\delta L}{\delta w_{ij}^{(k)}}$, а затем вычитаем эти производные из $w_{ij}^{(k)}$. Таким образом мы минимизируем функцию ошибки $L$.  
1. Возьмем претренированную сверточную нейронную сеть и зафиксируем ее, т.е. мы больше не собираемся в ней ничего менять. Прогоним оригинальное изображение и посчитаем матрицы Грама $G_{ij}^l$ на каждом слое.    
2. Сгенерируем изображение $I$ из случайного шума. Это изображение мы теперь будем улучшать, чтобы оно стало похожим на исходное изображение.
3. Прогоним сгенерированное изображение через сеть, так же посчитаем матрицы Грэма $\hat{G}_{ij}^l$ на каждом слое.   
4. Определим функцию ошибки, как взвешенную сумму $L_2^2$ расстояний между матрицами Грама оригинального изображения и соответствующими матрицами сгенерированного изображения.
$$E_l = \sum_{i,j} (G_{ij}^l - \hat{G}_{ij}^l)^2$$
$$L = \sum_l w_l E_l$$, где $w_l$ - действтельные числа, веса, с которыми ошибка на каждом слое войдет в общую ошибку. Это гиперпараметры, т.е. задаются до тренировки.
5. Посчитаем $\frac{\delta L}{\delta I_{ij}}$, где $I_{ij}$ - пиксели изображения $I$. Обновим изображение $I_{ij} = I_{ij} - \alpha \frac{\delta L}{\delta I_{ij}}$ ($\alpha$ - learning rate).    
6.  Проделаем 3, 4, 5 пока не получим желаемую текстуру. Результат будет хранится в $I$.

Все это проиллюстрировано и так же расписано на изображении ниже.

Замечания к изображению:
1. Здесь формула ошибки изображена только на последнем слое, но лосс считается на нескольких слоях, которые мы сами выбираем. Интуиция следующая: слои в начале сети будут сохранять низкоуровневые признаки, такие как, напрмер, грани, а те, что ближе к концу, будут говорить о структуре изображения в целом.

<img src='images/texture_gen_scheme.png'>
<div style="text-align: right"><font size="2">Изображение из статьи [1]</font></div>

Восстановление текстуры из разных слоёв извлекает разные признаки оригинального изображения
<img src='images/differ_layers.jpg'>

## Style transfer

Сгенерировав текстуру изображения можно теперь попытаться наложить эту текстуру другое изображение. Основным изменением является то, что теперь нам необходимо так же ещё считать функцию потерь с тем изображением, от которого мы хотим взять контент.
<img src='images/style_stranfer1.jpg'>

К функции ошибки, которая считала потери в "стиле", добавим часть, которая будут отвечать за оригинал, а именно контент изображения:
$$E_{content,k} = w_{content} \sum_{i,j} (F_{content}^k - F_{gen}^k)^2\,\,\,\,\,(1)$$, где $F_{content}^k$ - сохраненная карта признаков оригинального изображения $O$ на слое $k$, $F_{gen}^k$ - карта признаков генерируемого изображения на слое $k$, $w_{content}$ - вес изображения $O$ в финальном лоссе. Тогда:
$$L = (\sum_l w_{style,l} E_{style, l}) + E_{content,k}$$, где $k$ - номер слоя из которого будет извлекаться представление изображения $O$.

Оптимизируем до тех пор, пока не получим приемлимый результат
<img src='images/style_stranfer2.jpg'>

### Дополнительные изменения и улучшения

Изменяя соотношение весов для контента и стиля в функции потерь можно получать смещение в соответствующую сторону
<img src='images/style_stranfer3.jpg'>

Если брать среднее взвешенное матриц Грама нескольких изображений, то можно получать смешение нескольких стилей в один
<img src='images/style_stranfer4.jpg'>

#### Проблема:
Это всё хорошо, но у нас слишком много проходов по нейронной сети!

#### Решение
Обучим отдельную сеть, которая выполняет перенос стилей за нас!

### Fast Style Transfer
1. Тренируем feedforward network для каждого отдельного стиля
1. Используем предобученную CNN для подсчёта ошибки
1. После обучения получаем стилизованное изображение всего за один прямой проход
1. Также используем Instance Normalization вместо Batch Normalization 
<img src='images/style_stranfer5.jpg'>

Ссылки:   
1. [Статья на arxiv](https://arxiv.org/pdf/1505.07376.pdf)
2. [Презентация cs231n](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture12.pdf)
3. [Реализация на ...](https://paperswithcode.com/task/style-transfer)